## Setting up basic Graph



In [5]:
from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv
from typing import TypedDict
from langchain_core.tools import tool
import gradio as gr

In [12]:
load_dotenv(override=True)

True

In [26]:
import importlib
import os
from typing import Any, Optional

from langchain_core.tools import tool

import leadfinder.providers.base as provider_base
import leadfinder.providers.google_places as provider_google

importlib.reload(provider_base)
importlib.reload(provider_google)

from leadfinder.providers.base import Anchor, RawCandidate
from leadfinder.providers.google_places import GooglePlacesConfig, GooglePlacesProvider


async def google_places_search_async(
    query: str,
    *,
    center_lat: Optional[float] = None,
    center_lng: Optional[float] = None,
    radius_km: Optional[float] = None,
    max_results: int = 5,
) -> list[RawCandidate]:
    api_key = os.getenv("GOOGLE_PLACES_API_KEY")
    if not api_key:
        return [
            RawCandidate(
                source="google_places",
                source_id="",
                payload={"error": "GOOGLE_PLACES_API_KEY is not set"},
                name="",
                address_full="",
                city=None,
                region=None,
                country=None,
                lat=None,
                lng=None,
                phone=None,
                website_url=None,
                categories=[],
            )
        ]

    cfg = GooglePlacesConfig(api_key=api_key)
    if center_lat is None or center_lng is None or radius_km is None:
        anchor = None
    else:
        anchor = Anchor(
            center_lat=center_lat,
            center_lng=center_lng,
            radius_km=radius_km,
            quota=max_results,
        )

    async with GooglePlacesProvider(cfg) as provider:
        candidates, _ = await provider.search(query, anchor)

    return candidates[: max(1, max_results)]


@tool
def google_maps_search(
    query: str,
    center_lat: Optional[float] = None,
    center_lng: Optional[float] = None,
    radius_km: Optional[float] = None,
    max_results: int = 5,
) -> list[dict[str, Any]]:
    """Search Google Places using the provider implementation."""
    import asyncio

    results = asyncio.run(
        google_places_search_async(
            query,
            center_lat=center_lat,
            center_lng=center_lng,
            radius_km=radius_km,
            max_results=max_results,
        )
    )

    businesses = []
    for item in results:
        businesses.append(
            {
                "name": item.name,
                "address": item.address_full,
                "place_id": item.source_id,
                "types": item.categories,
                "phone": item.phone,
                "website": item.website_url,
                "lat": item.lat,
                "lng": item.lng,
            }
        )

    return businesses

In [27]:
import json

sample_results = await google_places_search_async(
    "pharmacies in Vancouver, Canada",
    max_results=10,
)

print(json.dumps([r.__dict__ for r in sample_results], indent=2))

[
  {
    "source": "google_places",
    "source_id": "ChIJUVFDWytyhlQRB5nvG1vemCo",
    "payload": {
      "search_place": {
        "id": "ChIJUVFDWytyhlQRB5nvG1vemCo",
        "types": [
          "pharmacy",
          "gift_shop",
          "drugstore",
          "health",
          "store",
          "point_of_interest",
          "establishment"
        ],
        "formattedAddress": "1125 Davie St, Vancouver, BC V6E 1N2",
        "location": {
          "latitude": 49.28098,
          "longitude": -123.13150200000001
        },
        "displayName": {
          "text": "Shoppers Drug Mart"
        }
      },
      "details": {
        "id": "ChIJUVFDWytyhlQRB5nvG1vemCo",
        "types": [
          "pharmacy",
          "drugstore",
          "gift_shop",
          "store",
          "health",
          "point_of_interest",
          "establishment"
        ],
        "nationalPhoneNumber": "(604) 669-2424",
        "formattedAddress": "1125 Davie St, Vancouver, BC V6E 1N2, Ca

In [28]:

import json


class ChatState(TypedDict):
    messages: list[dict]


def agent(state: ChatState) -> ChatState:
    messages = state["messages"]
    user_msg = next((m for m in reversed(messages) if m["role"] == "user"), None)
    query = user_msg["content"] if user_msg else ""

    tool_output = google_maps_search.invoke({"query": query, "max_results": 5})
    assistant_text = json.dumps(tool_output, indent=2)

    messages = messages + [{"role": "assistant", "content": assistant_text}]
    return {"messages": messages}


graph_builder = StateGraph(ChatState)
graph_builder.add_node("agent", agent)
graph_builder.add_edge(START, "agent")
graph_builder.add_edge("agent", END)
graph = graph_builder.compile()

In [31]:

import json


def respond(message: str, history: list[dict]) -> tuple[list[dict], str]:
    messages: list[dict] = list(history)
    messages.append({"role": "user", "content": message})

    state = graph.invoke({"messages": messages})
    assistant_text = state["messages"][-1]["content"]
    messages.append({"role": "assistant", "content": assistant_text})

    debug_text = json.dumps(messages, indent=2)
    print(debug_text)
    return messages, debug_text


with gr.Blocks() as demo:
    gr.Markdown("# LeadFinder Mock Chat")
    chatbot = gr.Chatbot(height=320)
    msg = gr.Textbox(placeholder="Ask for leads, e.g. 'Plumbers in Chicago'")
    clear = gr.Button("Clear")
    debug = gr.Textbox(label="Debug messages", lines=10, interactive=False)

    msg.submit(respond, [msg, chatbot], [chatbot, debug])
    clear.click(lambda: ([], ""), None, [chatbot, debug])


demo.launch(inline=True)

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/zoranjeremic/Sources/SourceLearningAI/leadfinder_service/.venv/lib/python3.12/site-packages/gradio/queueing.py", line 766, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/zoranjeremic/Sources/SourceLearningAI/leadfinder_service/.venv/lib/python3.12/site-packages/gradio/route_utils.py", line 355, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/zoranjeremic/Sources/SourceLearningAI/leadfinder_service/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 2163, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/zoranjeremic/Sources/SourceLearningAI/leadfinder_service/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 1940, in 